## Ejemplo de Tiny-tropue para un ejecicio tendencia de compra usando población Colombia

In [ ]:
import json
import sys
import os
import pandas as pd
import matplotlib.pyplot as plt

sys.path.insert(0, '..')

import tinytroupe
from tinytroupe import config_manager
from tinytroupe.agent import TinyPerson
from tinytroupe.environment import TinyWorld
from tinytroupe.factory import TinyPersonFactory
from tinytroupe.validation import TinyPersonValidator
from tinytroupe import control

from tinytroupe.extraction import ResultsExtractor
from tinytroupe.profiling import Profiler

from tinytroupe.validation import SimulationExperimentEmpiricalValidator, SimulationExperimentDataset, validate_simulation_experiment_empirically

from google.colab import userdata

os.environ['OPENAI_API_KEY']=userdata.get('openai_api')


In [ ]:
target_nationality = "Colombia"
population_size = 50

with open ("/content/drive/MyDrive/info_colombia/colombia_extra_info.txt", "r") as f:
    additional_demographic_specification = f.read()

In [ ]:
compute_other_nationalities = True

interviewer_introduction = """
Hola, soy parte de un equipo que estudia cómo las personas toman decisiones de compra en línea.
Queremos conocerte un poco mejor. ¿Puedes contarnos quién eres, cómo sueles comprar en internet,
y qué tipo de productos buscas más a menudo? También cuéntanos tus intereses, estilo de vida,
y si te consideras más práctico o más emocional al momento de comprar.
"""

interviewer_main_question = """
Imagina que estás buscando comprar una lavadora. Encuentras dos opciones idénticas:

1. En una tienda poco conocida (más barata), sin garantía clara, sin calificaciones ni respaldo.
2. En Mercado Libre (más costosa), pero con beneficios como entrega garantizada, pago a cuotas sin interés, devolución fácil, y respaldo de un vendedor con excelente reputación.

¿Cuál elegirías y por qué? Por favor, responde con tu elección (Opción 1 u Opción 2) y explica tus motivos con detalle.
Queremos entender cómo piensas al tomar decisiones como esta.
"""

inner_monologue = """
Voy a pensar con honestidad, como si esta situación me pasara de verdad.
Consideraré mi presupuesto, mis experiencias pasadas, mis preferencias personales, y la importancia que le doy
a la seguridad al comprar. No me limitaré a responder de forma neutral: si tengo una opinión clara, la diré.
También seré coherente con mi estilo de vida, nivel de confianza en las tiendas online, y cómo valoro los beneficios
extra frente al precio. Pensaré primero… y luego hablaré con sinceridad.
"""




In [ ]:
results_extractor = ResultsExtractor(
    extraction_objective=(
        "Determinar cuál opción de compra prefiere cada persona: "
        "Opción 1 (más barata sin respaldo) u Opción 2 (más costosa con beneficios de Mercado Libre)."
    ),
    situation=(
        "El agente fue consultado sobre cuál oferta preferiría al comprar una lavadora: "
        "una opción más barata sin respaldo (Opción 1), o una más costosa con beneficios como garantía, entrega, y reputación (Opción 2). "
        "El agente debía indicar su elección entre '1' o '2' y justificarla."
    ),
    fields=["name", "response", "justification"],
    fields_hints={
        "response": (
            "Debe ser exactamente '1', '2' o 'N/A' (si no se puede determinar la elección). "
            "No aceptes otros formatos como 'uno' o 'dos'."
        )
    },
    verbose=True
)

In [ ]:
# Función de analísis de resultados
def analizar_resultados_simulacion(df):
    """
    Analiza las respuestas simuladas sobre preferencias de compra entre dos opciones:
    Opción 1: Más barata, sin respaldo.
    Opción 2: Más costosa, con beneficios (Mercado Libre).
    """

    # Asegurar que la columna de respuesta sea string
    df["response"] = df["response"].astype(str)

    # Contar respuestas
    counts = df["response"].value_counts()
    total = counts.sum()

    porcentaje_opcion1 = counts.get("1", 0) / total
    porcentaje_opcion2 = counts.get("2", 0) / total
    porcentaje_na = counts.get("N/A", 0) / total

    # Mostrar análisis
    print("📊 Resultados de la Simulación de Preferencia de Compra")
    print("-----------------------------------------------------")
    print(f"Total de respuestas válidas: {total}")
    print(f"Opción 1 (más barata sin respaldo): {porcentaje_opcion1:.2%}")
    print(f"Opción 2 (más costosa con beneficios): {porcentaje_opcion2:.2%}")
    print(f"Respuestas no válidas o ambiguas (N/A): {porcentaje_na:.2%}")
    print()

    # Veredicto simple basado en proporción mayoritaria
    if porcentaje_opcion2 > porcentaje_opcion1:
        print("✅ Veredicto: La mayoría de personas simuladas prefiere la oferta con respaldo (Mercado Libre).")
    elif porcentaje_opcion1 > porcentaje_opcion2:
        print("⚠️ Veredicto: La mayoría de personas simuladas prefiere la opción más barata sin respaldo.")
    else:
        print("🤔 Veredicto: Las preferencias están divididas o no hay diferencia clara.")

In [ ]:
factory = TinyPersonFactory.create_factory_from_demography("/content/drive/MyDrive/info_colombia/colombia.json",
                                                           population_size=population_size,
                                                           additional_demographic_specification=additional_demographic_specification)


Contrucción de audiencia


In [ ]:
people = factory.generate_people(population_size, verbose=True)


In [ ]:
factory.sampling_dimensions

In [ ]:
###  Estadisticas de la población creada
profiler = Profiler()
profiler.profile(people)
### Para guardar en memria los datos creados de la población
control.checkpoint()

# Ambiente para la audiencia sin discusiones entre ellos:

audiencia = TinyWorld(f"Target audience ({target_nationality})", people, broadcast_if_no_target=False)



Desarrollo del experimento

In [ ]:
audiencia.broadcast(interviewer_introduction)
audiencia.broadcast(interviewer_main_question)
audiencia.broadcast_thought(inner_monologue)

In [ ]:
## Generación de las disertaciones
audiencia.run(2)

## Control de memoria
control.checkpoint()



Resultados

In [ ]:
results = results_extractor.extract_results_from_agents(people)

filtered_results = [item for item in results if isinstance(item, dict)]
## Concersion de los resultados
df = pd.DataFrame(filtered_results)
df["response"].value_counts()

Graficas de los rultados